# Ailerons norm
> Regresión

In [38]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
from functools import reduce

# https://xgboost.readthedocs.io/en/latest/python/python_api.html
import xgboost as xgb

# Cargar mi repo como paquete
from sklearn.datasets import load_boston
import sys
from os.path import abspath

try:
    from src.gaparsimony import GAparsimony
except:
    sys.path.append(abspath("C:/Users/Millan/Desktop/GAparsimony"))
    from src.gaparsimony import GAparsimony

In [39]:
df = pd.read_csv("../data/ailerons_norm.csv")
print(df.shape)
df.head()

(13750, 41)


,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,diffDiffClb,SeTime1,SeTime2,SeTime3,SeTime4,SeTime5,SeTime6,SeTime7,SeTime8,SeTime9,SeTime10,SeTime11,SeTime12,SeTime13,SeTime14,diffSeTime1,diffSeTime2,diffSeTime3,diffSeTime4,diffSeTime5,diffSeTime6,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se,goal
0,0.500512,0.222798,0.463320,0.387931,0.440000,0.550000,0.60,0.685185,0.664516,0.358025,0.303030,0.303030,0.303030,0.303030,0.303030,0.303030,0.303030,0.312500,0.312500,0.322581,0.322581,0.322581,0.322581,0.327869,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.3125,0.327869,0.750000
1,0.740266,0.310881,0.598456,0.568966,0.270000,0.416667,0.55,0.648148,0.696774,0.296296,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.343750,0.343750,0.354839,0.354839,0.354839,0.354839,0.360656,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.3125,0.360656,0.694444
2,0.584016,0.533679,0.644788,0.586207,0.400000,0.583333,0.60,0.490741,0.619355,0.353909,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.343750,0.343750,0.354839,0.354839,0.354839,0.354839,0.360656,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.3750,0.360656,0.666667
3,0.441598,0.538860,0.544402,0.560345,0.493333,0.616667,0.65,0.509259,0.587097,0.362140,0.318182,0.318182,0.318182,0.318182,0.318182,0.318182,0.318182,0.328125,0.328125,0.338710,0.338710,0.338710,0.338710,0.344262,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.3125,0.344262,0.694444
4,0.288934,0.404145,0.525097,0.525862,0.583333,0.416667,0.60,0.574074,0.651613,0.366255,0.272727,0.272727,0.272727,0.272727,0.272727,0.272727,0.272727,0.281250,0.281250,0.290323,0.290323,0.290323,0.290323,0.327869,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.307692,0.75,0.3125,0.327869,0.777778


In [40]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

data_train = pd.DataFrame(X_train, columns=df.columns[:-1]).reset_index(drop=True)
data_test = pd.DataFrame(X_test, columns=df.columns[:-1]).reset_index(drop=True)

print(f"train: {data_train.shape} test: {data_test.shape}")

train: (11000, 40) test: (2750, 40)


In [41]:
data_train.head()

,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,diffDiffClb,SeTime1,SeTime2,SeTime3,SeTime4,SeTime5,SeTime6,SeTime7,SeTime8,SeTime9,SeTime10,SeTime11,SeTime12,SeTime13,SeTime14,diffSeTime1,diffSeTime2,diffSeTime3,diffSeTime4,diffSeTime5,diffSeTime6,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se
0,0.598873,0.398964,0.467181,0.448276,0.256667,0.450000,0.85,0.583333,0.670968,0.378601,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.093750,0.093750,0.096774,0.096774,0.096774,0.096774,0.098361,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.0625,0.098361
1,0.248463,0.393782,0.428571,0.560345,0.630000,0.450000,0.55,0.546296,0.574194,0.370370,0.287879,0.287879,0.287879,0.287879,0.287879,0.287879,0.287879,0.296875,0.296875,0.306452,0.306452,0.354839,0.354839,0.360656,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.181818,0.666667,0.461538,0.75,0.3125,0.360656
2,0.426230,0.398964,0.644788,0.508621,0.623333,0.583333,0.50,0.648148,0.664516,0.337449,0.515152,0.515152,0.515152,0.515152,0.515152,0.515152,0.515152,0.531250,0.531250,0.548387,0.548387,0.548387,0.548387,0.557377,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.5625,0.557377
3,0.529201,0.445596,0.698842,0.551724,0.306667,0.583333,0.85,0.472222,0.593548,0.353909,0.121212,0.121212,0.121212,0.121212,0.121212,0.121212,0.121212,0.125000,0.125000,0.129032,0.129032,0.129032,0.129032,0.131148,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.1250,0.131148
4,0.657787,0.502591,0.671815,0.431034,0.203333,0.416667,0.75,0.629630,0.670968,0.358025,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.093750,0.093750,0.064516,0.064516,0.064516,0.064516,0.065574,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.888889,0.5,0.545455,0.666667,0.461538,0.75,0.0625,0.065574


In [42]:
data_test.head()

,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,diffDiffClb,SeTime1,SeTime2,SeTime3,SeTime4,SeTime5,SeTime6,SeTime7,SeTime8,SeTime9,SeTime10,SeTime11,SeTime12,SeTime13,SeTime14,diffSeTime1,diffSeTime2,diffSeTime3,diffSeTime4,diffSeTime5,diffSeTime6,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se
0,0.447234,0.419689,0.567568,0.474138,0.350000,0.716667,0.85,0.564815,0.548387,0.366255,0.090909,0.075758,0.075758,0.075758,0.075758,0.075758,0.075758,0.078125,0.078125,0.080645,0.080645,0.096774,0.096774,0.114754,0.666667,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.363636,0.666667,0.307692,0.75,0.0625,0.114754
1,0.231557,0.248705,0.471042,0.163793,0.453333,0.383333,0.60,0.611111,0.606452,0.720165,0.030303,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.046875,0.046875,0.048387,0.048387,0.048387,0.048387,0.049180,0.444444,0.5,0.6,0.5,0.636364,1.0,0.333333,0.5,0.555556,0.5,0.545455,0.666667,0.461538,0.75,0.0625,0.049180
2,0.488217,0.336788,0.718147,0.551724,0.363333,0.916667,0.60,0.527778,0.516129,0.341564,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.171875,0.171875,0.177419,0.177419,0.177419,0.177419,0.180328,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.1250,0.180328
3,0.526639,0.274611,0.745174,0.491379,0.296667,0.633333,0.65,0.601852,0.567742,0.362140,0.090909,0.136364,0.136364,0.136364,0.136364,0.136364,0.136364,0.140625,0.140625,0.145161,0.145161,0.129032,0.129032,0.098361,0.111111,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.636364,0.666667,0.538462,0.75,0.0625,0.098361
4,0.554816,0.559585,0.548263,0.491379,0.273333,0.766667,0.90,0.555556,0.541935,0.353909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.093750,0.093750,0.096774,0.096774,0.096774,0.096774,0.098361,0.555556,0.5,0.6,0.5,0.727273,1.0,0.555556,0.5,0.666667,0.5,0.545455,0.666667,0.461538,0.75,0.0625,0.098361


In [47]:
def fitness_XGBoost(chromosome):
    # First two values in chromosome are 'C' & 'sigma' of 'svmRadial' method
    tuneGrid = {
                "n_estimators ": int(chromosome[0]),
                "max_depth": int(chromosome[1]),
                "min_child_weight": int(chromosome[2]),
                "reg_alpha": chromosome[3],
                "reg_lambda": chromosome[4],
                "subsample": chromosome[5],
                "colsample_bytree": chromosome[6],
                "learning_rate": 0.01,
                "random_state": 1234,
                "verbosity": 0}
    
    # Next values of chromosome are the selected features (TRUE if > 0.50)
    selec_feat = chromosome[7:]>0.50
    
    # Return -Inf if there is not selected features
    if np.sum(selec_feat)<1:
        return np.array([np.NINF, np.NINF, np.Inf])
    
    # Extract features from the original DB plus response (last column)
    data_train_model = data_train.loc[: , data_train.columns[selec_feat]] 
    data_test_model = data_test.loc[: , data_test.columns[selec_feat]] 
    
    # How to validate each individual
    # 'repeats' could be increased to obtain a more robust validation metric. Also,
    # 'number' of folds could be adjusted to improve the measure.
    train_control = RepeatedKFold(n_splits=10, n_repeats=5, random_state=123)

    # train the model
#     np.random.seed(1234)

    aux = xgb.XGBRegressor(**tuneGrid)

    model = cross_val_score(aux, data_train_model, y_train, scoring="neg_mean_squared_error", cv=train_control, n_jobs=-1)

    

    # Extract kappa statistics (the repeated k-fold CV and the kappa with the test DB)
    rmse_val = model.mean()

    model = xgb.XGBRegressor(**tuneGrid).fit(data_train_model, y_train)

    rmse_test = mean_squared_error(model.predict(data_test_model), y_test)
    # Obtain Complexity = Num_Features*1E6+Number of support vectors
    complexity = np.sum(selec_feat)*1E6 + len(model.get_booster().get_dump())
    
    # Return(validation score, testing score, model_complexity)
    return np.array([rmse_val, -rmse_test, complexity])

In [48]:
# Ranges of size and decay
min_param = np.array([10, 2, 1, 0., 0., 0.6, 0.8])
max_param = np.array([2000, 20, 20, 1., 1., 1., 1.])
names_param = ["n_estimators(nrounds)","max_depth", "min_child_weight", 
               "reg_alpha(lasso)", "reg_lambda(ridge)", "subsample",
               "colsample_bytree"]

# ga_parsimony can be executed with a different set of 'rerank_error' values
rerank_error = 0.01

In [51]:
GAparsimony_model = GAparsimony(fitness=fitness_XGBoost,
                                  min_param=min_param,
                                  max_param=max_param,
                                  names_param=names_param,
                                  nFeatures=len(df.columns[:-1]),
                                  names_features=df.columns[:-1],
                                  keep_history = True,
                                  rerank_error = rerank_error,
                                  popSize = 64,
                                  elitism = 16,
                                  maxiter = 100, early_stop=10,
                                  feat_thres=0.90, # Perc selected features in first generation
                                  feat_mut_thres=0.10, # Prob of a feature to be one in mutation
                                  parallel = True, seed_ini = 1234,
                                  verbose=GAparsimony.MONITOR)

In [52]:
GAparsimony_model.fit()


Step 0. Initial population
[[           nan            nan            nan 6.58154753e+02
  1.90929074e+01 8.36116887e+00 1.17343791e-01 6.37838172e-01
  8.28036705e-01 8.40695418e-01 1.00000000e+00 0.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 0.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [           nan            nan            nan 1.33399487e+03
  1.27104221e+01 1.40062919e+00 5.50694639e-01 1.28306152e-02
  7.35176626e-01 8.30750064e-01 1.00000000e+00 1.00000000e+00
  1.00000

KeyboardInterrupt: 

In [ ]:
GAparsimony_model.summary()

In [ ]:
GAparsimony_model.plot()